In [24]:
import numpy as np
import pickle

In [26]:
# Get symmetrys from pdb file
# This function returns the dictionary of symmetry operation

def Symmetry_of_pdb(fname):
    """
    Read REMARK 350 BIOMT from pdb file, which specify the necessary transformation to get the full protein structure.
    Returns the dictionary of symmetry operation.
    """
    fin = open(fname, 'r')

    sym_dict = {}  # dict to save the symmetry rotations and chain id
    trans_dict = {}  # dict to save the symmetry translations and chain id
    atom_count = 0
    line = fin.readline()
    while line:

        # read symmetry transformations
        flag1 = 'REMARK 350 APPLY THE FOLLOWING TO CHAINS: '
        flag2 = 'REMARK 350                    AND CHAINS: '
        if line.startswith(flag1):
            line = line.strip()
            chainIDs = line.replace(flag1, '').replace(',', '').split()
            line = fin.readline().strip()
            while line.startswith(flag2):
                chainIDs += line.replace(flag2, '').replace(',', '').split()
                line = fin.readline().strip()
            sys_tmp = []
            trans_tmp = []
            while line[13:18] == 'BIOMT':
                sys_tmp.append([float(line[24:33]), float(line[34:43]), float(line[44:53])])
                trans_tmp.append(float(line[58:68]))
                line = fin.readline().strip()
            sym_dict[tuple(chainIDs)] = np.asarray(sys_tmp)  # cannot use list as dict keys, but tuple works
            trans_dict[tuple(chainIDs)] = np.asarray(trans_tmp)
            print chainIDs
            
            continue

        line = fin.readline()
    fin.close()

    ##################################################################################################################
    # if no REMARK 350 provided, then save atoms_dict in atoms directly
    if not sym_dict.keys():
        print "no 350 found"
        return None
    
    ##################################################################################################################
    # Deal with the case where we have remark 350
    
    rotation_matrix_dic = {}
    transition_matrix_dic = {}
    
    for chainIDs in sym_dict.keys():
        
        rotation_matrix_dic.update({"Chain:"+str(chainIDs) : {}})
        transition_matrix_dic.update({"Chain:"+str(chainIDs) : {}})

        sym_array = sym_dict[chainIDs]
        trans_array = trans_dict[chainIDs]
        for i in range(int(len(sym_array) / 3)):
            sym_op = sym_array[3 * i:3 * (i + 1), :]
            trans = trans_array[3 * i:3 * (i + 1)]

            rotation_matrix_dic["Chain:"+str(chainIDs)].update({"No.%d"%i:sym_op})
            transition_matrix_dic["Chain:"+str(chainIDs)].update({"No.%d"%i:trans})
            
    return rotation_matrix_dic, transition_matrix_dic
    #return atom_info, sym_dict, atoms_array

In [27]:
symmetrys, translations = Symmetry_of_pdb('../pysingfel_test/tests/input/1UF2.pdb')

['A', 'B', 'P', 'C', 'D', 'Q', 'E', 'F', 'R', 'G', 'H', 'S', 'I', 'J', 'T', 'K']


In [28]:
Icosahedron_Symmetry = symmetrys["Chain:('A', 'B', 'P', 'C', 'D', 'Q', 'E', 'F', 'R', 'G', 'H', 'S', 'I', 'J', 'T', 'K')"]

In [29]:
with open('./symmetry_zoo/Icosahedron_Symmetry.pkl','w') as target:
    pickle.dump(Icosahedron_Symmetry, target)